<a href="https://colab.research.google.com/github/natanrajch/DiploDatos/blob/main/MELI/Hito_4_de_Mentoria_MELI_nDCG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hito 4 - Grupo B - MELI Data Challenge 2020 - DiploDatos 2021




## Dependencies y carga de datos

In [ ]:
!pip install fasttext

In [2]:
import fasttext
import gzip
import pandas as pd
import urllib
import tarfile
import urllib.request
import numpy as np
import random
import json
import bisect
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import scipy.sparse as sps

url_item_data = "https://meli-data-challenge.s3.amazonaws.com/2020/item_data.jl.gz"
url_train_data = "https://meli-data-challenge.s3.amazonaws.com/2020/train_dataset.jl.gz"
print('train')
train_data = []
with urllib.request.urlopen(url_train_data) as handle:
  gz = gzip.GzipFile(fileobj=handle)
  for i, line in enumerate(gz):
    train_data.append(json.loads(line.strip().decode('utf-8')))

train_df = pd.DataFrame(train_data)
# train_df.user_history
# print('item')
# item_data = []
# with urllib.request.urlopen(url_item_data) as handle:
#   gz = gzip.GzipFile(fileobj=handle)
#   for i, line in enumerate(gz):
#     item_data.append(json.loads(line.strip().decode('utf-8')))

# item_df = pd.DataFrame(item_data)
del train_data
# del item_data
del gz
# print('training')

train


## Entrenamiento modelo FastText train_supervised

In [3]:
## armo el xtrain
oracion=[]
for row in train_df.itertuples():
    item_label='__label__'+str(row.item_bought)
    item=''
    for i in range(0,len(row.user_history)):
        if row.user_history[i]['event_type']=='view': # asi accedo x ejemplo a la primera tupla de mi user_history que contiene toda la info de un evento. 
          item=str(((row.user_history)[i]['event_info']))+' '+item # Y asi al valor asociado a una key en particular. 
        else:
           item=str(((row.user_history)[i]['event_info'])).split(sep=' ')[0].lower()+' '+item # Y asi al valor asociado a una key en particular.  
    total_item=item_label+' '+item
    oracion.append(total_item)
    
## armo el ytrain
target=[]
for row in train_df.itertuples():
    target.append(str(row.item_bought))

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

x_train,x_test=train_test_split(oracion, test_size=0.20, random_state=4)


In [5]:
with open('train_file_80_s&v.txt', 'w', encoding="utf-8") as f:
    for item in x_train:
        f.write("%s\n" % item)
      

In [6]:
with open('test_file_20_s&v.txt', 'w', encoding="utf-8") as f:
    for item in x_test:
        f.write("%s\n" % item)

In [ ]:
model=fasttext.train_supervised(input='train_file_80_s&v.txt')

## Carga de modelo FastText train_supervised pre-entrenado

In [7]:
!gdown https://drive.google.com/u/0/uc?id=1tzOLc2nSYn6VyC35kctYCJh4Rb0DdFIk&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=1tzOLc2nSYn6VyC35kctYCJh4Rb0DdFIk
To: /content/model_fasttext_80_s&v_lr05_epch15.bin
605MB [00:05, 105MB/s]


In [8]:
import fasttext
model2=fasttext.load_model('/content/model_fasttext_80_s&v_lr05_epch15.bin')
#model.test('test_file_20_s&v.txt') #devuelve precision & recall. 

In [9]:
model2.test("/content/test_file_20_s&v.txt")

(77310, 0.09693441986806364, 0.09693441986806364)

In [27]:
item_data = []
with urllib.request.urlopen(url_item_data) as handle:
  gz = gzip.GzipFile(fileobj=handle)
  for i, line in enumerate(gz):
    item_data.append(json.loads(line.strip().decode('utf-8')))

item_df = pd.DataFrame(item_data)
del gz
del item_data

### nDCG en datos de entrenamiento

In [62]:
item_list = item_df['item_id'].tolist()
domain_list = item_df['domain_id'].tolist()
item_domain_dict = dict(zip(item_list, domain_list))

In [98]:
#Se arregla un unico valor que estaba trayendo problemas
x_train[85731] = '__label__1595373 108992 108992 216650 locao 1651058 1651058 1651058 locao 1866033 locao 1065548 locao 1651058 locao 2011467 locao 1651058 locao 2011467 locao clean 180427 180427 180427 180427 692954 1436635 450809 celular 1849196 celular celular celular celular celular celular celular celular celular celular celular celular celular celular 1234585 1234585 celular celular celular celular celular celular 171910 171910 171910 celular 1234585 1234585 1234585 1234585 celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular celular 839876 839876 839876 celular celular 441855 441855 celular celular celular celular '

In [99]:
true_bitem = []
pred_y_fastt_model = []

for row in x_train:
  tb = int(row[:20].split(' ')[0][9:])
  true_bitem.append(tb)

  preds = []
  model_preds = model2.predict(row.split(' ',1)[1],k=10)[0]
  for pred in model_preds:
    preds.append(int(pred[9:]))
  pred_y_fastt_model.append(preds)


In [100]:
#CALCULO DEL nDCG
def ndcg(predictions,bitem):
  ''' Dada una List de predictions, y un item comprado (int), devuelve el calculo de nDCG de la prediccion'''
  best = 22.4246159748234
  current = 0
  bdom = item_domain_dict[bitem]
  for i in range(1,11):
    pred = predictions[i-1]
    if pred == bitem:
      gain = 12
    elif item_domain_dict[pred] == bdom:
      gain = 1
    else:
      gain = 0
    current += (1/np.log(1+i)) * gain
  return current/best 

# true_bitem = train_df.item_bought.tolist()
ndcg_list = []
for i in range(len(true_bitem)):
  ndcg_list.append(ndcg(pred_y_fastt_model[i], true_bitem[i]))

In [101]:
print('nDCG obtenido en datos de ENTRENAMIENTO:' , sum(ndcg_list) / len(ndcg_list))

nDCG obtenido en datos de ENTRENAMIENTO: 0.7416162377983386


In [102]:
df_ndcg = pd.DataFrame(ndcg_list, columns=['value'])
df_ndcg.describe()

,value
count,330530.000000
mean,0.741616
std,0.221736
min,0.000000
25%,0.772024
50%,0.772024
75%,0.839774
max,1.000000


### nDCG en datos de test

In [82]:
true_bitem = []
pred_y_fastt_model = []

for row in x_test:
  tb = int(row[:20].split(' ')[0][9:])
  true_bitem.append(tb)

  preds = []
  model_preds = model2.predict(row.split(' ',1)[1],k=10)[0]
  for pred in model_preds:
    preds.append(int(pred[9:]))
  pred_y_fastt_model.append(preds)



In [84]:
#CALCULO DEL nDCG
def ndcg(predictions,bitem):
  ''' Dada una List de predictions, y un item comprado (int), devuelve el calculo de nDCG de la prediccion'''
  best = 22.4246159748234
  current = 0
  bdom = item_domain_dict[bitem]
  for i in range(1,11):
    pred = predictions[i-1]
    if pred == bitem:
      gain = 12
    elif item_domain_dict[pred] == bdom:
      gain = 1
    else:
      gain = 0
    current += (1/np.log(1+i)) * gain
  return current/best 

# true_bitem = train_df.item_bought.tolist()
ndcg_list = []
for i in range(len(true_bitem)):
  ndcg_list.append(ndcg(pred_y_fastt_model[i], true_bitem[i]))

In [90]:
print('nDCG obtenido en datos de TEST:' , sum(ndcg_list) / len(ndcg_list))

nDCG obtenido en datos de TEST: 0.1463826505146321


In [89]:
df_ndcg = pd.DataFrame(ndcg_list, columns=['value'])
df_ndcg.describe()

,value
count,82633.000000
mean,0.146383
std,0.271168
min,0.000000
25%,0.000000
50%,0.000000
75%,0.153151
max,1.000000
